In [2]:
from torchvision import transforms
import random
import torch
import numpy as np

# Define las transformaciones
resize = transforms.Resize((112, 112))
horizontal_flip = transforms.RandomHorizontalFlip(p=0.5)
rotation = transforms.RandomRotation(degrees=10)
color_jitter = transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1)
random_crop = transforms.RandomResizedCrop(size=(112, 112), scale=(0.9, 1.0), ratio=(0.9, 1.1))
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Función para aplicar transformaciones
def apply_transform_list(imgs, train=True):
    # Seed para asegurarse de que las transformaciones aleatorias sean consistentes
    seed = np.random.randint(2147483647)
    random.seed(seed)
    torch.manual_seed(seed)

    # Generar parámetros aleatorios para las transformaciones
    params = {
        'horizontal_flip': random.random(),
        'rotation': random.uniform(-10, 10),
        'brightness': random.uniform(0.9, 1.1),
        'contrast': random.uniform(0.9, 1.1),
        'saturation': random.uniform(0.9, 1.1),
        'hue': random.uniform(-0.1, 0.1),
        'crop_params': random_crop.get_params(resize(imgs[0]), scale=(0.9, 1.0), ratio=(0.9, 1.1))
    }

    new_imgs = []

    for img in imgs:
        img = resize(img)
        
        if train:
            # Aplicar transformaciones solo en entrenamiento
            if params['horizontal_flip'] < 0.5:
                img = transforms.functional.hflip(img)
            img = transforms.functional.rotate(img, params['rotation'])
            img = transforms.functional.adjust_brightness(img, params['brightness'])
            img = transforms.functional.adjust_contrast(img, params['contrast'])
            img = transforms.functional.adjust_saturation(img, params['saturation'])
            img = transforms.functional.adjust_hue(img, params['hue'])
            img = transforms.functional.resized_crop(img, *params['crop_params'], size=(112, 112))

        img = to_tensor(img)
        img = normalize(img)

        new_imgs.append(img)
    
    return new_imgs

In [3]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import glob
import random

class FireSeriesDataset(Dataset):
    def __init__(self, root_dir, img_size=112, transform=None, train=True):
        self.transform = transform
        self.sets = glob.glob(f"{root_dir}/**/*")
        self.img_size = img_size
        self.train = train  # Para diferenciar entre entrenamiento y validación/prueba
        random.shuffle(self.sets)

    def __len__(self):
        return len(self.sets)

    def __getitem__(self, idx):
        img_folder = self.sets[idx]
        img_list = glob.glob(f"{img_folder}/*.jpg")

        # Cargar todas las imágenes disponibles
        images = [Image.open(file) for file in img_list]

        # Redimensionar todas las imágenes al tamaño img_size
        img_list = [im.resize((self.img_size, self.img_size)) for im in images]

        # Aplicar las transformaciones a las imágenes (solo si train=True)
        tensor_list = apply_transform_list(img_list, self.train)

        # Concatenar todas las imágenes en un solo tensor
        return torch.cat(tensor_list, dim=0), int(img_folder.split("/")[-2])

In [4]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class FireDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=16, img_size=112, num_workers=12):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        self.train_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "train"), self.img_size, train=True
        )
        self.val_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "val"), self.img_size, train=False
        )
        self.test_dataset = FireSeriesDataset(
            os.path.join(self.data_dir, "test"), self.img_size, train=False
        )

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchmetrics import Accuracy, Precision, Recall
import pytorch_lightning as pl

class FireClassifier(pl.LightningModule):
    def __init__(self, learning_rate=1e-4, hidden_dim=256, lstm_layers=2):
        super(FireClassifier, self).__init__()
        self.save_hyperparameters()

        # Usar ResNet18 como extractor de características
        self.feature_extractor = models.resnet18(pretrained=True)
        self.feature_extractor.fc = nn.Identity()  # Remover la última capa de ResNet
        # salida 512
        # LSTM para procesar las secuencias de características
        self.lstm = nn.LSTM(
            input_size=512,  # Tamaño de las características de ResNet
            hidden_size=hidden_dim,  # Tamaño oculto de la LSTM
            num_layers=lstm_layers,
            batch_first=True
        )

        # Capa final para la clasificación binaria
        self.fc = nn.Linear(hidden_dim, 1)

        # Métricas
        self.train_accuracy = Accuracy(task="binary")
        self.val_accuracy = Accuracy(task="binary")
        self.test_accuracy = Accuracy(task="binary")
        self.train_precision = Precision(task="binary")
        self.val_precision = Precision(task="binary")
        self.test_precision = Precision(task="binary")
        self.train_recall = Recall(task="binary")
        self.val_recall = Recall(task="binary")
        self.test_recall = Recall(task="binary")

    def forward(self, x):
        batch_size, sequence_len, C, H, W = x.shape  # Asumimos secuencia de imágenes
        x = x.view(batch_size * sequence_len, C, H, W)  # Aplanar para pasar por ResNet

        # Extraer características con ResNet
        features = self.feature_extractor(x)

        # Reorganizar las características en secuencias para la LSTM
        features = features.view(batch_size, sequence_len, -1)

        # Pasar las características por la LSTM
        lstm_out, (hn, cn) = self.lstm(features)

        # Usar la última salida de la LSTM para la clasificación
        output = self.fc(lstm_out[:, -1, :])

        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        acc = self.train_accuracy(torch.sigmoid(y_hat), y.int())
        precision = self.train_precision(torch.sigmoid(y_hat), y.int())
        recall = self.train_recall(torch.sigmoid(y_hat), y.int())
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        self.log("train_precision", precision)
        self.log("train_recall", recall)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        acc = self.val_accuracy(torch.sigmoid(y_hat), y.int())
        precision = self.val_precision(torch.sigmoid(y_hat), y.int())
        recall = self.val_recall(torch.sigmoid(y_hat), y.int())
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        self.log("val_precision", precision)
        self.log("val_recall", recall)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = F.binary_cross_entropy_with_logits(y_hat, y.float())
        acc = self.test_accuracy(torch.sigmoid(y_hat), y.int())
        precision = self.test_precision(torch.sigmoid(y_hat), y.int())
        recall = self.test_recall(torch.sigmoid(y_hat), y.int())
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        self.log("test_precision", precision)
        self.log("test_recall", recall)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams['learning_rate'], weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler
        }

In [22]:
from pytorch_lightning import Trainer

# Crear el DataModule y el modelo
data_module = FireDataModule(data_dir=r"C:\nico\wildfire2024\data\temporal_ds\images", batch_size=16, img_size=112)
model = FireClassifier(learning_rate=1e-4)

# Entrenador de PyTorch Lightning
trainer = Trainer(max_epochs=10) # Ajusta el número de GPUs si es necesario

# Entrenar el modelo
trainer.fit(model, data_module)

# Evaluar el modelo en el conjunto de prueba
trainer.test(model, datamodule=data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

   | Name              | Type            | Params | Mode 
---------------------------------------------------------------
0  | feature_extractor | ResNet          | 11.2 M | train
1  | lstm              | LSTM            | 1.3 M  | train
2  | fc                | Linear          | 257    | train
3  | train_accuracy    | BinaryAccuracy  | 0      | train
4  | val_accuracy      | BinaryAccuracy  | 0      | train
5  | test_accuracy     | BinaryAccuracy  | 0      | train
6  | train_precision   | BinaryPrecision | 0      | train
7  | val_precision     | BinaryPrecision | 0      | train
8  | test_precision    | BinaryPrecision | 0      | train
9  | train_recall      | BinaryRecall    | 0      | train
10 | val_recall        | BinaryRecall    | 0      | train
11 | test_recall       | BinaryRecall    | 0      | train
---------------------------------------------------------------
12.5 M

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 19020, 12480, 21124, 19224, 1792, 1344, 14976, 15244, 12092, 19688, 9620, 16336) exited unexpectedly

In [23]:
import torch
print(torch.cuda.is_available())  # Verifica si CUDA está disponible
print(torch.cuda.device_count()) 

False
0


In [24]:
# dimensiones de la entrada
test = torch.randn(20, 3, 112, 112)
out = a(test)
out.shape

Tamaño del dataset de entrenamiento: 906
Tamaño del dataset de validación: 108
Tamaño del dataset de prueba: 120


RuntimeError: DataLoader worker (pid(s) 17636, 5660, 21452, 8380, 20512, 21188, 20248, 19432, 16724, 20828, 21460, 19668) exited unexpectedly

In [37]:
def check_images_in_folders(root_path):
    all_folders_correct = True
    for folder in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder)
        if os.path.isdir(folder_path):
            images = [file for file in os.listdir(folder_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
            count = len(images)
            if count != 10:
                print(f"La carpeta {folder} tiene {count} imágenes, se esperaban 10.")
                all_folders_correct = False
            else:
                # print(f"La carpeta {folder} tiene exactamente 10 imágenes.")
                pass
    
    if all_folders_correct:
        print("Todas las carpetas contienen exactamente 10 imágenes.")
    else:
        print("Algunas carpetas no contienen el número correcto de imágenes.")

# Usar la función, especificando la ruta del directorio raíz
root_directory = r"C:\nico\wildfire2024\data\temporal_ds\images\val\0"

check_images_in_folders(root_directory)
root_directory = r"C:\nico\wildfire2024\data\temporal_ds\images\val\1"

check_images_in_folders(root_directory)
root_directory = r"C:\nico\wildfire2024\data\temporal_ds\images\train\0"
check_images_in_folders(root_directory)
root_directory = r"C:\nico\wildfire2024\data\temporal_ds\images\train\1"
check_images_in_folders(root_directory)
root_directory = r"C:\nico\wildfire2024\data\temporal_ds\images\test\0"
check_images_in_folders(root_directory)
root_directory = r"C:\nico\wildfire2024\data\temporal_ds\images\test\1"
check_images_in_folders(root_directory)

Todas las carpetas contienen exactamente 10 imágenes.
Todas las carpetas contienen exactamente 10 imágenes.
Todas las carpetas contienen exactamente 10 imágenes.
Todas las carpetas contienen exactamente 10 imágenes.
Todas las carpetas contienen exactamente 10 imágenes.
Todas las carpetas contienen exactamente 10 imágenes.
